In [2]:
# 数据预处理
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

print("=" * 50)
print("AIS数据预处理")
print("=" * 50)

AIS数据预处理


In [4]:
# 1. 加载数据
try:
    df = pd.read_csv("../data/raw/AIS_2023_01_01.csv", nrows=50000)
    print(f" 加载 {len(df)} 条数据")
except:
    print("❌ 数据文件不存在")
    exit()

# 2. 数据清洗
print("\n1. 数据清洗...")
original_len = len(df)

# 删除经纬度缺失值
df = df.dropna(subset=['LAT', 'LON'])
print(f"  删除LAT/LON缺失值: {original_len} -> {len(df)}")

# 过滤无效经纬度
df = df[(df['LAT'].between(-90, 90)) & (df['LON'].between(-180, 180))]
print(f"  过滤无效经纬度: {len(df)} 条")

# 过滤异常速度
if 'SOG' in df.columns:
    df = df[df['SOG'] <= 50]
    print(f"  过滤速度异常值(SOG>50): {len(df)} 条")


 加载 50000 条数据

1. 数据清洗...
  删除LAT/LON缺失值: 50000 -> 50000
  过滤无效经纬度: 50000 条
  过滤速度异常值(SOG>50): 49873 条


In [5]:
# 3. 特征选择（只保留聚类需要的字段）
print("\n2. 特征选择...")
clustering_cols = ['LAT', 'LON']
df_clustering = df[clustering_cols].copy()
print(f"  选择特征: {clustering_cols}")

# 4. 数据标准化
print("\n3. 数据标准化...")
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df_clustering)
df_scaled = pd.DataFrame(X_scaled, columns=['LAT_scaled', 'LON_scaled'])

print(f"  标准化前 LAT范围: [{df_clustering['LAT'].min():.2f}, {df_clustering['LAT'].max():.2f}]")
print(f"  标准化后 LAT范围: [{df_scaled['LAT_scaled'].min():.2f}, {df_scaled['LAT_scaled'].max():.2f}]")

# 5. 保存处理后的数据
print("\n4. 保存数据...")
# 合并原始数据和处理后数据
df_processed = pd.concat([df.reset_index(drop=True), df_scaled], axis=1)

# 保存完整数据
df_processed.to_csv("../data/processed/processed_data.csv", index=False)
print(f" 保存完整处理数据到: ../data/processed/processed_data.csv")


2. 特征选择...
  选择特征: ['LAT', 'LON']

3. 数据标准化...
  标准化前 LAT范围: [15.60, 49.78]
  标准化后 LAT范围: [0.00, 1.00]

4. 保存数据...
 保存完整处理数据到: ../data/processed/processed_data.csv


In [6]:
# 保存不同规模的数据用于实验
for size in [1000, 5000, 10000, 50000]:
    if len(df_processed) >= size:
        sample = df_processed.sample(size, random_state=42)
        sample.to_csv(f"../data/processed/data_{size}.csv", index=False)
        print(f" 保存 {size} 条样本数据")

print("\n 数据预处理完成！")

 保存 1000 条样本数据
 保存 5000 条样本数据
 保存 10000 条样本数据

 数据预处理完成！
